# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [43]:
!pip install gmaps

In [44]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [45]:

weather_data = pd.read_csv("world_cities.csv")
weather_data

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Gamba,100,GA,1613426905,83,-2.6500,10.0000,78.926,1.14
1,1,Kavieng,46,PG,1613426905,74,-2.5744,150.7967,84.218,3.76
2,2,Bredasdorp,97,ZA,1613426905,67,-34.5322,20.0403,57.470,1.03
3,3,Longyearbyen,75,SJ,1613426905,100,78.2186,15.6401,26.870,1.03
4,4,Dayong,0,CN,1613426905,72,25.0209,118.2898,59.270,4.00
...,...,...,...,...,...,...,...,...,...,...
547,547,Honningsvåg,75,NO,1613426942,86,70.9821,25.9704,25.070,1.54
548,548,Payakumbuh,52,ID,1613426942,91,-0.2202,100.6308,69.908,0.59
549,549,Aranos,3,NaN,1613426942,26,-24.1333,19.1167,71.222,4.25
550,550,Maţāy,0,EG,1613426942,42,28.4190,30.7792,54.536,2.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [46]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [47]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:

# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
30,30,Butaritari,0,KI,1613426907,79,3.0707,172.7902,79.646,6.65
39,39,Saint-Philippe,0,RE,1613426907,69,-21.3585,55.7679,79.070,3.09
92,92,Loei,0,TH,1613426911,66,17.4167,101.5000,72.266,0.89
109,109,Salalah,0,OM,1613426913,60,17.0151,54.0924,70.070,1.03
121,121,Coyhaique,0,CL,1613426817,29,-45.5752,-72.0662,75.470,5.14
148,148,Sur,0,OM,1613426915,63,22.5667,59.5289,71.168,3.79
182,182,Bandar,0,ID,1613426918,93,2.0500,99.7500,70.862,0.67
211,211,Pisco,0,PE,1613426671,65,-13.7000,-76.2167,77.270,8.75
342,342,Dubti,0,ET,1613426928,32,11.7361,41.0853,77.864,4.51
357,357,Lázaro Cárdenas,0,MX,1613426711,57,17.9583,-102.2000,77.270,4.83


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
30,Butaritari,KI,3.0707,172.7902,
39,Saint-Philippe,RE,-21.3585,55.7679,
92,Loei,TH,17.4167,101.5000,
109,Salalah,OM,17.0151,54.0924,
121,Coyhaique,CL,-45.5752,-72.0662,
148,Sur,OM,22.5667,59.5289,
182,Bandar,ID,2.0500,99.7500,
211,Pisco,PE,-13.7000,-76.2167,
342,Dubti,ET,11.7361,41.0853,
357,Lázaro Cárdenas,MX,17.9583,-102.2000,


In [51]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [52]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City,Country,Lat,Lng,Hotel Name
30,Butaritari,KI,3.0707,172.7902,NaN
39,Saint-Philippe,RE,-21.3585,55.7679,NaN
92,Loei,TH,17.4167,101.5000,NaN
109,Salalah,OM,17.0151,54.0924,NaN
121,Coyhaique,CL,-45.5752,-72.0662,NaN
148,Sur,OM,22.5667,59.5289,NaN
182,Bandar,ID,2.0500,99.7500,NaN
211,Pisco,PE,-13.7000,-76.2167,NaN
342,Dubti,ET,11.7361,41.0853,NaN
357,Lázaro Cárdenas,MX,17.9583,-102.2000,NaN


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [54]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))